In [1]:
import numpy as np
import os
from PIL import Image
from PIL.Image import Resampling
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision import models
from torch import nn
from lion_pytorch import Lion

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import scipy.io

mat_data = scipy.io.loadmat('imagelabels.mat')
image_data = mat_data['labels']
image_array = np.array(image_data)[0]

In [3]:
size = 256


def resize_and_crop(img):
    x, y = img.size

    if x > y:
        scale = size / y
        new_x = int(x * scale)
        img_resized = img.resize((new_x, size), Resampling.LANCZOS)

        crop_margin = int(((size / 2) * (x - y)) / y)
        left = crop_margin
        right = left + size
        img_cropped = img_resized.crop((left, 0, right, size))

    elif y > x:
        scale = size / x
        new_y = int(y * scale)
        img_resized = img.resize((size, new_y), Resampling.LANCZOS)

        crop_margin = int(((size / 2) * (y - x)) / x)
        top = crop_margin
        bottom = crop_margin + size
        img_cropped = img_resized.crop((0, top, size, bottom))

    else:
        img_cropped = img.resize((size, size), Resampling.LANCZOS)

    return img_cropped.convert("RGB")

def resize_and_crop2(img):
    x, y = img.size

    if x > y:
        scale = size / y
        new_x = int(x * scale)
        img_resized = img.resize((new_x, size), Resampling.LANCZOS)

        crop_margin = int(((size / 1.1) * (x - y)) / y)
        left = crop_margin
        right = left + size
        img_cropped = img_resized.crop((left, 0, right, size))

    elif y > x:
        scale = size / x
        new_y = int(y * scale)
        img_resized = img.resize((size, new_y), Resampling.LANCZOS)

        crop_margin = int(((size / 1.1) * (y - x)) / x)
        top = crop_margin
        bottom = crop_margin + size
        img_cropped = img_resized.crop((0, top, size, bottom))

    else:
        img_cropped = img.resize((size, size), Resampling.LANCZOS)

    return img_cropped.convert("RGB")


class Dataset102f:
    def __init__(self):
        ds = []
        self.classes = 102

        self.one_hot_dict = {}
        for i in range(102):
            one_hot_np = np.zeros(102)
            one_hot_np[i] = 1
            one_hot_tensor = torch.from_numpy(one_hot_np).float().to(device)
            self.one_hot_dict[i] = one_hot_tensor

        for i in range(8189):
            offset = i + 1
            clazz = image_array[i]
            img = Image.open("102f/image_" + "{:05d}".format(offset) + ".jpg")

            img1 = resize_and_crop(img)
            np_img = np.array(img1).astype(np.float32) / 255
            tensor_img = torch.from_numpy(np_img).float().permute(2, 0, 1).to(device)
            ds.append((tensor_img, self.one_hot_dict[clazz - 1]))

            img2 = resize_and_crop2(img)
            np_img = np.array(img2).astype(np.float32) / 255
            tensor_img = torch.from_numpy(np_img).float().permute(2, 0, 1).to(device)
            ds.append((tensor_img, self.one_hot_dict[clazz - 1]))

        np.random.shuffle(ds)
        split_point = int(len(ds) * 0.8)
        self.train_data = ds[:split_point]
        self.test_data = ds[split_point:]


dataset = Dataset102f()

tds_train = TensorDataset(torch.stack([x[0] for x in dataset.train_data]),
                          torch.stack([x[1] for x in dataset.train_data]), )
tdsl_train = DataLoader(tds_train, batch_size=32, shuffle=True)

tds_test = TensorDataset(torch.stack([x[0] for x in dataset.test_data]),
                         torch.stack([x[1] for x in dataset.test_data]), )
tdsl_test = DataLoader(tds_test, batch_size=32, shuffle=False)

dataset.train_data = None  # free memory
torch.cuda.empty_cache()

In [4]:
from torchvision.models import EfficientNet_B0_Weights

model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

for param in model.parameters():
    param.requires_grad = False

in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, dataset.classes)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Lion(model.parameters(), lr=1e-4)

In [5]:
def train_epoch(model, data_loader, loss_fn, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for x, y in data_loader:
        # Forward pass
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs, y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Calculate statistics
        total_loss += loss.item()

        pred_labels = torch.argmax(outputs, dim=1)
        y_labels = torch.argmax(y, dim=1)
        correct += (pred_labels == y_labels).sum().item()
        total += y.size(0)

    acc = correct / total
    avg_loss = total_loss / len(data_loader)
    return avg_loss, acc


def evaluate_epoch(model, data_loader):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in data_loader:
            pred = model(x)

            pred_labels = torch.argmax(pred, dim=1)
            y_labels = torch.argmax(y, dim=1)

            correct += (pred_labels == y_labels).sum().item()

    accuracy = correct / len(data_loader.dataset)
    return total_loss / len(data_loader), accuracy

In [6]:
for epoch in range(30):
    train_loss, train_acc = train_epoch(model, tdsl_train, loss_fn, optimizer)
    test_loss, test_acc = evaluate_epoch(model, tdsl_test)
    print(f"Epoch [{epoch + 1}] - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, ")
    if test_acc > 0.975:
        torch.save(model, f"102f_model_{epoch + 1}.pth")

Epoch [1] - Train Loss: 2.8107, Train Acc: 0.5324, Test Loss: 0.0000, Test Acc: 0.8883, 
Epoch [2] - Train Loss: 0.7345, Train Acc: 0.9028, Test Loss: 0.0000, Test Acc: 0.9365, 
Epoch [3] - Train Loss: 0.3117, Train Acc: 0.9454, Test Loss: 0.0000, Test Acc: 0.9634, 
Epoch [4] - Train Loss: 0.1842, Train Acc: 0.9677, Test Loss: 0.0000, Test Acc: 0.9722, 
Epoch [5] - Train Loss: 0.1217, Train Acc: 0.9782, Test Loss: 0.0000, Test Acc: 0.9777, 
Epoch [6] - Train Loss: 0.0953, Train Acc: 0.9820, Test Loss: 0.0000, Test Acc: 0.9805, 
Epoch [7] - Train Loss: 0.0749, Train Acc: 0.9867, Test Loss: 0.0000, Test Acc: 0.9838, 
Epoch [8] - Train Loss: 0.0618, Train Acc: 0.9890, Test Loss: 0.0000, Test Acc: 0.9866, 
Epoch [9] - Train Loss: 0.0546, Train Acc: 0.9895, Test Loss: 0.0000, Test Acc: 0.9863, 
Epoch [10] - Train Loss: 0.0458, Train Acc: 0.9913, Test Loss: 0.0000, Test Acc: 0.9850, 
Epoch [11] - Train Loss: 0.0469, Train Acc: 0.9901, Test Loss: 0.0000, Test Acc: 0.9850, 
Epoch [12] - Train 